In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np

In [2]:
data = pd.read_csv('/Users/annafurgala-wojas/New project/Heart-disease-dataset/healthcare-dataset-stroke-data.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
data.groupby("stroke")["bmi"].apply(lambda x: x.isna().sum())

stroke
0    161
1     40
Name: bmi, dtype: int64

In [4]:
data["bmi"] = data.groupby("stroke")["bmi"].transform(
    lambda x: x.fillna(x.median())
)

In [5]:
data.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.871409,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.699951,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.800000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.000000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,32.800000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [6]:
data = data.drop(columns=["id"])

In [7]:

data_final = pd.get_dummies(data, columns=["gender", "ever_married", "work_type", "Residence_type", "smoking_status"], dtype=int)

data_final.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
1,61.0,0,0,202.21,29.7,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [8]:
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X = data_final.drop(columns=["stroke"])
y = data_final['stroke']

In [9]:
smt = SMOTETomek(sampling_strategy='auto', random_state=42)
X_res, y_res = smt.fit_resample(X, y)

print("The amount of samples SMOTETomek:")
print(pd.Series(y_res).value_counts())

The amount of samples SMOTETomek:
stroke
1    4821
0    4821
Name: count, dtype: int64


/Users/annafurgala-wojas/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/annafurgala-wojas/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/Users/annafurgala-wojas/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [10]:
X_train_tomek, X_test_tomek, y_train_tomek, y_test_tomek = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)

In [11]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF_tomek = RandomForestClassifier(
    n_estimators=100, 
    criterion='entropy', 
    random_state=0,
    class_weight='balanced' 
)
classifier_RF_tomek.fit(X_train_tomek, y_train_tomek)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       random_state=0)

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
y_pred_RF_tomek = classifier_RF_tomek.predict(X_test_tomek)
accuracy_score(y_test_tomek, y_pred_RF_tomek)
print(classification_report(y_test_tomek,y_pred_RF_tomek))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       965
           1       0.99      0.97      0.98       964

    accuracy                           0.98      1929
   macro avg       0.98      0.98      0.98      1929
weighted avg       0.98      0.98      0.98      1929

